# Two Layer Net

In [1]:
import sys, os

import numpy as np

sys.path.append(os.pardir)
from Data.common.functions import *
from Data.common.gradient import numerical_gradient

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, 
                 weight_init_std=0.01):
        # Initialization
        self.parms = {}
        self.parms['W1'] = (weight_init_std * 
                            np.random.randn(input_size, hidden_size))
        self.parms['b1'] = np.zeros(hidden_size)
        self.parms['W2'] = (weight_init_std *
                            np.random.randn(hidden_size, output_size))
        self.parms['b2'] = np.zeros(output_size)
        
    def predict(self, x):
        W1, W2 = self.parms['W1'], self.parms['W2']
        b1, b2 = self.parms['b1'], self.parms['b2']
        
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
    
    # x:input data, t:supervised data
    def loss(self, x, t):
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    # x:input data, t:supervised data
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.parms['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.parms['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.parms['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.parms['b2'])
        
        return grads

Example

In [2]:
net = TwoLayerNet(input_size=784, hidden_size=100, output_size=10)
net.parms['W1'].shape  # (784, 100)
net.parms['b1'].shape  # (100, )
net.parms['W2'].shape  # (100, 10)
net.parms['b2'].shape  # (10, )

(10,)

In [3]:
x = np.random.randn(100, 784)
y = net.predict(x)

print(x, y)

[[ 0.93355971  1.81330377  1.10522476 ...  1.300684   -0.06946814
   0.00790263]
 [-0.68416527 -0.25575427  1.62561394 ... -0.0815943   0.15976661
  -1.16795041]
 [-0.02522362  0.22759012  0.03006273 ...  0.95299973 -0.30842773
  -0.13670869]
 ...
 [-0.33497465 -0.57036121 -0.82067943 ... -1.11202925  0.8079055
  -0.75582429]
 [ 0.04287831  0.17515853 -1.20598381 ...  0.98240858  0.53983976
   2.04316759]
 [ 0.13823927 -1.03725688  0.70451859 ...  1.35503249 -1.50235947
   0.56810008]] [[0.10183856 0.10047881 0.09771114 0.10763618 0.10954496 0.09982238
  0.09048761 0.09017318 0.11183786 0.09046932]
 [0.10078394 0.10049247 0.09698397 0.10969162 0.10829953 0.09916909
  0.09004153 0.09087258 0.11161902 0.09204625]
 [0.10193418 0.10042756 0.09753037 0.10842869 0.10759443 0.09942182
  0.09091496 0.09114721 0.11086092 0.09173986]
 [0.10003145 0.10074041 0.09860755 0.10740731 0.10846876 0.10010079
  0.08987128 0.09034782 0.11239723 0.09202739]
 [0.0991108  0.10086383 0.09830252 0.10773203 0.1

In [4]:
x = np.random.randn(100, 784)
t = np.random.randn(100, 10)

grads = net.numerical_gradient(x, t)
print(grads['W1'].shape)
print(grads['b1'].shape)
print(grads['W2'].shape)
print(grads['b2'].shape)

(784, 100)
(100,)
(100, 10)
(10,)


# Implementation of Mini-batch

In [6]:
import numpy as np
from Data.dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

train_loss_list = []

# Hyperparameters
iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in range(iters_num):
    # Fetch mini-batch
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # Compute gradient
    grad = network.numerical_gradient(x_batch, t_batch)
    
    # Update parameters
    for key in ['W1', 'b1', 'W2', 'b2']:
        network.parms[key] -= learning_rate * grad[key]
        
    # Record the learning process
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

KeyboardInterrupt: 

In [5]:
import numpy as np
from Data.dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

train_loss_list = []
train_acc_list = []
test_acc_list = []
# Average repetitions per epoch
iter_per_epoch = max(train_size / batch_size, 1)

# Hyperparameters
iters_num = 10000
batch_size = 100
learning_rate = 0.1

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in range(iters_num):
    # Fetch mini-batch
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # Compute gradient
    grad = net.numerical_gradient(x_batch, t_batch)
    
    # Update Parameters
    for key in ['W1', 'b1', 'W2', 'b2']:
        network.parms[key] -= learning_rate * grad[key]
        
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    # Compute the accuracy per epoch
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc |" + str(train_acc) + str(test_acc))

NameError: name 'train_size' is not defined